In [1]:
# Libraries
import pandas as pd
import sqlite3 as lite

con = lite.connect('../data/election.db')
cur = con.cursor() 

In [3]:
# Load data
dept2014_xl = pd.ExcelFile('../data/input/Dep_15_Resultats_com_T1_c.xlsx')
dept2014_dfs = {sheet_name: dept2014_xl.parse(sheet_name) 
          for sheet_name in dept2014_xl.sheet_names}

Check Communes table

In [85]:
coms = dept2014_dfs['Feuil1'][['Libellé de la commune','Code de la commune', 'Code du département']]
cur.execute("SELECT * FROM Communes")
rows = cur.fetchall()
for row in rows[0:10]:
    print(row)

IntegrityError: UNIQUE constraint failed: Communes.CommId, Communes.DeptId

In [5]:
dept2014_dfs['Feuil1'].head(n =12)

,Date de l'export,Code du département,Libellé du département,Code du canton,Libellé du canton,Code de la commune,Libellé de la commune,Inscrits,Abstentions,% Abs/Ins,...,Binôme.9,Voix.9,% Voix/Ins.9,% Voix/Exp.9,N°Panneau.10,Nuance.10,Binôme.10,Voix.10,% Voix/Ins.10,% Voix/Exp.10
0,2015-03-23 15:52:51,1,AIN,1,Ambérieu-en-Bugey,2,L'Abergement-de-Varey,220,93,42.27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-03-23 15:52:51,1,AIN,1,Ambérieu-en-Bugey,4,Ambérieu-en-Bugey,8100,4327,53.42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-03-23 15:52:51,1,AIN,1,Ambérieu-en-Bugey,7,Ambronay,1737,836,48.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-03-23 15:52:51,1,AIN,1,Ambérieu-en-Bugey,8,Ambutrix,564,275,48.76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-03-23 15:52:51,1,AIN,1,Ambérieu-en-Bugey,13,Arandas,139,53,38.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-03-23 15:52:51,1,AIN,1,Ambérieu-en-Bugey,17,Argis,302,115,38.08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-03-23 15:52:52,1,AIN,1,Ambérieu-en-Bugey,41,Bettant,601,284,47.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015-03-23 15:52:52,1,AIN,1,Ambérieu-en-Bugey,89,Château-Gaillard,1264,641,50.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015-03-23 15:52:52,1,AIN,1,Ambérieu-en-Bugey,107,Cleyzieu,116,32,27.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015-03-23 15:52:52,1,AIN,1,Ambérieu-en-Bugey,111,Conand,105,51,48.57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
pres2012_T2 = pres2012_dfs['Tour 2'].head(n = 2)
# removing % columns
pres2012_T2 = pres2012_T2.drop([x for x in pres2012_T2.columns.values.tolist() if x.rfind('%')>-1],axis=1)
pres2012_T2 = pres2012_T2.drop(['Votants', 'Exprimés'],axis=1)

# Reshaping:
n_cand = len([x for x in pres2012_T2.columns.values.tolist() if x.rfind('Nom')>-1])

for i in range(0, n_cand):
    if i == 0:
        pres2012_T2.loc[:,'cand.0'] = pres2012_T2[['Sexe','Prénom','Nom']].apply(lambda x: '_'.join(x), axis=1)
        pres2012_T2.loc[:,'Voix.0'] = pres2012_T2['Voix'] 
        pres2012_T2.drop(['Sexe','Prénom','Nom','Voix'],inplace=True,axis=1)
    else:
        i_s = str(i)
        pres2012_T2.loc[:,'cand.'+i_s] = pres2012_T2[['Sexe.'+i_s,'Prénom.'+i_s,'Nom.'+i_s]].apply(lambda x: '_'.join(x), axis=1)
        pres2012_T2.drop(['Sexe.'+i_s,'Prénom.'+i_s,'Nom.'+i_s],inplace=True,axis=1)


pres2012_T2.head(n = 10)

,Code du département,Libellé du département,Code de la commune,Libellé de la commune,Inscrits,Abstentions,Blancs et nuls,Voix.1,cand.0,Voix.0,cand.1
0,1,AIN,1,L'Abergement-Clémenciat,592,86,31,280,M_François_HOLLANDE,195,M_Nicolas_SARKOZY
1,1,AIN,2,L'Abergement-de-Varey,215,40,18,75,M_François_HOLLANDE,82,M_Nicolas_SARKOZY


In [65]:
# Extract candidats names


In [63]:
pres2012_T1 = pres2012_dfs['Tour 1'].head(n = 2)
# removing % columns
pres2012_T1 = pres2012_T1.drop([x for x in pres2012_T1.columns.values.tolist() if x.rfind('%')>-1],axis=1)
pres2012_T1 = pres2012_T1.drop(['Votants', 'Exprimés'],axis=1)

# Reshaping:
n_cand = len([x for x in pres2012_T1.columns.values.tolist() if x.rfind('Nom')>-1])

for i in range(0, n_cand):
    if i == 0:
        pres2012_T1.loc[:,'cand.0'] = pres2012_T1[['Sexe','Prénom','Nom']].apply(lambda x: '_'.join(x), axis=1)
        pres2012_T1.loc[:,'Voix.0'] = pres2012_T1['Voix'] 
        pres2012_T1.drop(['Sexe','Prénom','Nom','Voix'],inplace=True,axis=1)
    else:
        i_s = str(i)
        pres2012_T1.loc[:,'cand.'+i_s] = pres2012_T1[['Sexe.'+i_s,'Prénom.'+i_s,'Nom.'+i_s]].apply(lambda x: '_'.join(x), axis=1)
        pres2012_T1.drop(['Sexe.'+i_s,'Prénom.'+i_s,'Nom.'+i_s],inplace=True,axis=1)


pres2012_T1.head(n = 10)

,Code du département,Libellé du département,Code de la commune,Libellé de la commune,Inscrits,Abstentions,Blancs et nuls,Voix.1,Voix.2,Voix.3,...,Voix.0,cand.1,cand.2,cand.3,cand.4,cand.5,cand.6,cand.7,cand.8,cand.9
0,1,AIN,1,L'Abergement-Clémenciat,592,84,9,126,159,25,...,13,F_Marine_LE PEN,M_Nicolas_SARKOZY,M_Jean-Luc_MÉLENCHON,M_Philippe_POUTOU,F_Nathalie_ARTHAUD,M_Jacques_CHEMINADE,M_François_BAYROU,M_Nicolas_DUPONT-AIGNAN,M_François_HOLLANDE
1,1,AIN,2,L'Abergement-de-Varey,215,36,5,38,45,15,...,10,F_Marine_LE PEN,M_Nicolas_SARKOZY,M_Jean-Luc_MÉLENCHON,M_Philippe_POUTOU,F_Nathalie_ARTHAUD,M_Jacques_CHEMINADE,M_François_BAYROU,M_Nicolas_DUPONT-AIGNAN,M_François_HOLLANDE
